<a href="https://colab.research.google.com/github/theAkhileshRai/Augmented-Reality/blob/master/AssetLvl_Pred_lNN_BinaryClassifierDCP03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

In [49]:
df = pd.read_csv('/content/DCP_leak_classification_train_data.csv',index_col=False)

In [50]:
df.head()

,Unnamed: 0,Unnamed: 0.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,0,2019-12-01 00:30:00,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238677,820.900548,1357.93250,Ashland_Unit256,0.0
1,1,1,2019-12-01 01:30:00,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,0.0
2,2,2,2019-12-01 02:30:00,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,0.0
3,3,3,2019-12-01 03:30:00,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,0.0
4,4,4,2019-12-01 04:30:00,1.0,109.567843,227.339240,351.392953,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,0.0


In [51]:
df['s_temp_mean*s_pressure_mean'] = df['s_pressure_mean']*df['s_temp_mean']
df['d_temp_mean*d_pressure_mean'] = df['d_pressure_mean']*df['d_temp_mean']

In [52]:
df['s_temp_crank*s_pressure_crank'] = df['s_temp_crank']*df['s_pressure_crank']
df['d_temp_crank*d_pressure_crank'] = df['d_pressure_crank']*df['d_temp_crank']

In [53]:
df.head()

,Unnamed: 0,Unnamed: 0.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,0,0,2019-12-01 00:30:00,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238677,820.900548,1357.93250,Ashland_Unit256,0.0,35495.693656,178966.511725,36568.496381,181230.701138
1,1,1,2019-12-01 01:30:00,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,0.0,34287.262878,176957.908115,35230.156167,179385.590149
2,2,2,2019-12-01 02:30:00,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,0.0,33953.168178,177511.639310,34687.769611,179829.918400
3,3,3,2019-12-01 03:30:00,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,0.0,36123.580593,185232.534887,36900.189112,187543.369582
4,4,4,2019-12-01 04:30:00,1.0,109.567843,227.339240,351.392953,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,0.0,37697.983703,191259.228686,38501.367766,193516.619885


In [54]:
# df1 = pd.read_csv('/content/old_normalised_assets-2.csv',index_col=False)
# df2 = pd.read_csv('/content/new_normalised_assets-2.csv',index_col=False)

In [55]:
df['Asset'].unique()

array(['Ashland_Unit256', 'Center station_C2', 'Crossbar_Unit1',
       'El Dorado_Unit', 'Goliad_Unit1100', 'HobitLP_Unit102',
       'Hobit_Unit2', 'Katy_Unit1116', 'Katy_Unit1117', 'LOHT_North',
       'La Porte_K11A', 'Lightning_Unit2 (2)(1).csv', 'RJ_Unit1(1).csv',
       'Lightning_Unit4', 'Reagon_Unit3', 'Saltlake_K71010B',
       'Strawberry_Unit1', 'Reagon_Unit6', 'West Turkey_Unit2'],
      dtype=object)

In [56]:
# 'Clark county_6', 'RJ booster_1', 'Strawberry_3.csv'

In [57]:
df2 = df.loc[(df.Asset == 'Clark county_6') | (df.Asset == 'RJ booster_1') | (df.Asset == 'Strawberry_3')|(df.Asset == 'Lightning_Unit2')]

In [58]:
df2.shape

(0, 20)

In [59]:
df2["Binary"].unique()

array([], dtype=float64)

In [60]:
#  
#        
#        

In [61]:
df1 = df.loc[(df.Asset == 'West_Turkey_Unit2') | (df.Asset == 'Cross bar_Unit1') | (df.Asset == 'Reagan_1_Unit6') | (df.Asset == 'Reagan_1_Unit3')|
        (df.Asset == 'Hobit_Unit2')|(df.Asset == 'Katy_Unit1116')|(df.Asset == 'Katy_Unit1117')|
       (df.Asset == 'HobitLP_Unit102')|(df.Asset == 'Goliad_Unit1100')|(df.Asset == 'Lightning_Unit4')|(df.Asset == 'Ashland_Unit256') | 
       (df.Asset == 'Center station_C2') | (df.Asset == 'El Dorado_Unit J2502') | (df.Asset == 'Saltlake_K71010B')]

In [62]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,0,0,2019-12-01 00:30:00,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238677,820.900548,1357.93250,Ashland_Unit256,0.0,35495.693656,178966.511725,36568.496381,181230.701138
1,1,1,2019-12-01 01:30:00,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,0.0,34287.262878,176957.908115,35230.156167,179385.590149
2,2,2,2019-12-01 02:30:00,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,0.0,33953.168178,177511.639310,34687.769611,179829.918400
3,3,3,2019-12-01 03:30:00,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,0.0,36123.580593,185232.534887,36900.189112,187543.369582
4,4,4,2019-12-01 04:30:00,1.0,109.567843,227.339240,351.392953,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,0.0,37697.983703,191259.228686,38501.367766,193516.619885


In [63]:
df1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'time', 'cyl', 's_temp_crank',
       'd_temp_crank', 's_pressure_crank', 'd_pressure_crank', 'rpm_crank',
       's_temp_mean', 'd_temp_mean', 's_pressure_mean', 'd_pressure_mean',
       'rpm_mean', 'Asset', 'Binary', 's_temp_mean*s_pressure_mean',
       'd_temp_mean*d_pressure_mean', 's_temp_crank*s_pressure_crank',
       'd_temp_crank*d_pressure_crank'],
      dtype='object')

In [64]:
df1['Binary'] = df1['Binary'].fillna(0)
#df1['multi_class'] = df1['multi_class'].fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,0,0,2019-12-01 00:30:00,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238677,820.900548,1357.93250,Ashland_Unit256,0.0,35495.693656,178966.511725,36568.496381,181230.701138
1,1,1,2019-12-01 01:30:00,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,0.0,34287.262878,176957.908115,35230.156167,179385.590149
2,2,2,2019-12-01 02:30:00,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,0.0,33953.168178,177511.639310,34687.769611,179829.918400
3,3,3,2019-12-01 03:30:00,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,0.0,36123.580593,185232.534887,36900.189112,187543.369582
4,4,4,2019-12-01 04:30:00,1.0,109.567843,227.339240,351.392953,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,0.0,37697.983703,191259.228686,38501.367766,193516.619885


In [66]:
df1.shape

(95569, 20)

In [67]:
df1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'time', 'cyl', 's_temp_crank',
       'd_temp_crank', 's_pressure_crank', 'd_pressure_crank', 'rpm_crank',
       's_temp_mean', 'd_temp_mean', 's_pressure_mean', 'd_pressure_mean',
       'rpm_mean', 'Asset', 'Binary', 's_temp_mean*s_pressure_mean',
       'd_temp_mean*d_pressure_mean', 's_temp_crank*s_pressure_crank',
       'd_temp_crank*d_pressure_crank'],
      dtype='object')

In [68]:
X_columns = [ 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',\
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',\
       's_pressure_mean', 'd_pressure_mean','s_temp_mean*s_pressure_mean','d_temp_mean*d_pressure_mean', 'rpm_mean']

In [69]:
len(X_columns)

13

In [70]:
X = df1[X_columns]

In [71]:
X.shape

(95569, 13)

# build  with mean

In [72]:
X

,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,rpm_mean
0,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238677,820.900548,35495.693656,178966.511725,1357.93250
1,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,34287.262878,176957.908115,1358.24000
2,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,33953.168178,177511.639310,1398.28125
3,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,36123.580593,185232.534887,1397.87125
4,1.0,109.567843,227.339240,351.392953,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,37697.983703,191259.228686,1397.31000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117803,2.0,146.551041,260.017853,883.423667,1783.572442,356.84550,146.551041,260.017853,876.475395,1783.204623,128448.381898,463665.037757,356.84550
117804,2.0,147.870794,260.943809,882.021744,1782.027466,356.84550,147.870794,260.943809,876.575532,1782.360609,129619.920145,465095.965660,356.84550
117805,2.0,148.115587,261.454681,881.335088,1778.050584,356.68650,148.115587,261.454681,875.617075,1777.167773,129692.537108,464648.832635,356.68650
117806,2.0,148.637102,261.667544,880.619821,1777.020600,356.72875,148.637102,261.667544,876.489700,1776.581254,130278.889238,464873.653634,356.72875


In [73]:
y_true = df1.iloc[:,15]

In [74]:
y_true.unique()

array([0., 1.])

In [75]:
#seed = 42
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size,shuffle= True)

In [76]:
# X_val, X_test, y_val, y_test = train_test_split(X_Test, y_Test, test_size=0.75)

In [77]:
from collections import Counter

In [78]:
print(X_train.shape,y_train.shape)

(81233, 13) (81233,)


In [79]:
len(X_test)

14336

In [80]:
#len(X_val)

In [81]:
Counter(y_train)

Counter({0.0: 57982, 1.0: 23251})

64->16->4->1

In [82]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 15.
        self.layer_1 = nn.Linear(13, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 64)
        self.layer_4 = nn.Linear(64, 10)
        self.layer_5 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(10)

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [83]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [84]:
X_test

array([[-1.22068951,  0.30938121,  0.08266454, ...,  0.0774405 ,
         0.1372636 , -0.13922631],
       [ 1.92921235, -0.60196962, -0.35277505, ..., -0.34711763,
        -0.08075318,  0.13149203],
       [-1.22068951, -0.59538395, -0.95637733, ..., -0.01913527,
        -0.51753119, -0.26567849],
       ...,
       [-0.17072222, -0.53589013,  1.14027291, ..., -1.35571353,
        -0.93414249,         nan],
       [-0.17072222, -0.48569189, -0.90087256, ...,  0.04956264,
        -0.38516491, -0.03401202],
       [-1.22068951,  0.27539585,  0.13750902, ...,  0.02526948,
         0.20500215, -0.13771959]])

In [85]:
EPOCHS = 100
BATCH_SIZE = 75
LEARNING_RATE = 0.0792

In [86]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

y_train = y_train.values
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
# ## val data    
# class valData(Dataset):
    
#     def __init__(self, X_data):
#         self.X_data = X_data
        
#     def __getitem__(self, index):
#         return self.X_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

test_data = testData(torch.FloatTensor(X_test))
# val_data = valData(torch.FloatTensor(X_val))

In [87]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset=val_data, batch_size=1)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [88]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [89]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=13, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [90]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [91]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [92]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        #X_batch = X_batch
  
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

In [93]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [95]:
y_pred_list

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [94]:
confusion_matrix(y_test, y_pred_list)

ValueError: ignored

# Old Assets

In [ ]:
print(classification_report(y_test, y_pred_list))

In [ ]:
import pickle
filename = 'binarynn_dcp_model.pkl'

In [ ]:
torch.save(model.state_dict(),filename)


In [ ]:
model.load_state_dict(torch.load(filename))

In [ ]:
model.eval()

# Test Pred New Assets

In [ ]:
X = df2[X_columns]

In [ ]:
y_true = df2.iloc[:,14]

In [ ]:
X_test = scaler.fit_transform(X)

In [ ]:
test_data = testData(torch.FloatTensor(X_test))

In [ ]:
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
X_test.shape

In [ ]:
y_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())



# New Assets

In [ ]:
predictions=[]
predictions.append([y_pred_list[i][0][0] for i in range(len(y_pred_list))])

In [ ]:
print(classification_report(y_true, predictions[0]))